In [1]:
import numpy as np
import pandas as pd
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.optim import lr_scheduler
from torch.optim import lr_scheduler
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

def normalization(x: list):
    M, m = np.max(x), np.min(x)
    for i in range(len(x)):
        x[i] = (x[i] - (M + m) / 2) / ((M - m) / 2)
    # x in [-1, 1]
    return M, m, x

def ArrNorm(x: np.ndarray):
    assert isinstance(x, np.ndarray), "We need list"
    M_list, m_list, res = [], [], []
    for i in range(x.shape[0]):
        u = x[i].tolist()
        M, m, t = normalization(u)
        res.append(t)
        M_list.append(M)
        m_list.append(m)
    return M_list, m_list, np.array(res)


def df2arr(x) -> np.ndarray:
    return np.array(x, dtype=np.float32)


In [2]:
excel = pd.read_excel('./data/A32.xlsx', header=None)
excel.shape

(4459, 18)

In [3]:
sp = [1486, 2972, 4458]
station_1 = excel.iloc[1:sp[0]+1,1:6]
station_2 = excel.iloc[sp[0]+1:sp[1]+1,1:6]
standard = excel.iloc[sp[1]+1:sp[2]+1,1:6]
standard

,1,2,3,4,5
2973,16,33,15,1.5,74
2974,16,33,15,1.7,74
2975,16,38,16,1.8,76
2976,18,41,21,1.9,79
2977,17,41,23,1.7,78
...,...,...,...,...,...
4454,16,23,49,21,99
4455,18,26,55,20.7,99
4456,19,28,52,20.8,99
4457,22,31,41,21,98


In [4]:
station_1 = df2arr(station_1)
station_2 = df2arr(station_2)
standard = df2arr(standard)
station_1.shape, station_2.shape, standard.shape

((1486, 5), (1486, 5), (1486, 5))

In [5]:
s1_minus_sd = station_1 - standard
s2_minus_sd = station_2 - standard
s1_div_sd = station_1 / standard
s2_dic_sd = station_2 / standard

C:\Users\Chen Ding\AppData\Local\Temp\ipykernel_30284\711113420.py:3: RuntimeWarning: divide by zero encountered in divide
  s1_div_sd = station_1 / standard
C:\Users\Chen Ding\AppData\Local\Temp\ipykernel_30284\711113420.py:4: RuntimeWarning: divide by zero encountered in divide
  s2_dic_sd = station_2 / standard


In [6]:
s1_minus_sd.shape, s2_minus_sd.shape

((1486, 5), (1486, 5))

In [7]:
s1_minus_sd_M, s1_minus_sd_m, s1_minus_sd = ArrNorm(s1_minus_sd)

In [8]:
def GetDataset(input_arr: list, output_arr: list, seq: int):
    assert(len(input_arr)==len(output_arr)), "Different size of input and output!"
    Input = []
    Output = []
    for i in range(input_arr.shape[0]-seq):
        Input.append(input_arr[i:i+seq][:])
        Output.append(output_arr[i:i+seq][:])
    return torch.tensor(Input, dtype=torch.float32), torch.tensor(Output, dtype=torch.float32)

        
def load_array(data_arrays, batch_size, is_train=True):
    # data-iter
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


In [9]:
s1_minus_sd_M

[10.0,
 10.0,
 8.0,
 6.0,
 5.0,
 2.299999952316284,
 2.200000047683716,
 8.0,
 13.0,
 15.0,
 14.0,
 16.0,
 14.0,
 14.0,
 12.0,
 10.0,
 13.0,
 16.0,
 8.0,
 10.0,
 16.0,
 21.0,
 20.0,
 18.0,
 17.0,
 17.0,
 20.0,
 14.0,
 13.0,
 19.0,
 20.0,
 20.0,
 10.0,
 12.0,
 17.0,
 14.0,
 11.0,
 11.0,
 19.0,
 10.0,
 14.0,
 16.0,
 14.0,
 14.0,
 18.0,
 3.0,
 27.0,
 34.0,
 37.0,
 24.0,
 15.0,
 12.0,
 13.0,
 4.0,
 3.0,
 4.0,
 5.0,
 5.0,
 17.0,
 23.0,
 3.0,
 3.0,
 4.0,
 7.0,
 12.0,
 15.0,
 15.0,
 8.0,
 4.0,
 2.3000011444091797,
 5.0,
 2.200000762939453,
 2.0999984741210938,
 2.1999988555908203,
 2.200000762939453,
 2.299999237060547,
 2.200000762939453,
 2.0,
 1.8999996185302734,
 5.0,
 9.0,
 7.0,
 6.0,
 8.0,
 7.0,
 3.0,
 22.0,
 34.0,
 25.0,
 27.0,
 21.0,
 18.0,
 7.0,
 7.0,
 4.0,
 15.0,
 26.0,
 12.0,
 1.8999996185302734,
 1.9000005722045898,
 1.8999996185302734,
 1.7999992370605469,
 1.8999996185302734,
 3.0,
 11.0,
 4.0,
 18.0,
 12.0,
 2.8000011444091797,
 3.0,
 3.1000003814697266,
 11.0,
 9.0,
 9.0,
 4.0

In [10]:
# one train demo
Input_Data_1, Output_Data_1 = GetDataset(s1_minus_sd, standard, 15)



C:\Users\Chen Ding\AppData\Local\Temp\ipykernel_30284\3545116846.py:8: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  return torch.tensor(Input, dtype=torch.float32), torch.tensor(Output, dtype=torch.float32)
